In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler, MultiLabelBinarizer
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
import re
from sklearn.linear_model import LinearRegression
from fancyimpute import IterativeImputer
from sklearn.neighbors import NearestNeighbors
from sklearn.impute import KNNImputer
from sklearn.metrics import confusion_matrix, accuracy_score
import appmode
from keras.models import model_from_json
from IPython.display import display, Markdown

In [2]:
symptom_cd_dummies = pd.read_csv('symptom_cd_dummies.csv')
del symptom_cd_dummies['Unnamed: 0']
symptom_cd_dummies

,age,gender,acute respiratory distress,asymptomatic,cold chills,cough,diarrhea,fatigue,fever,headache,...,dyslipidemia,enlarged prostate,heart disease,hepatitis B,hypertension,kidney disease,none,parkinson's disease,thyroid disease,didDie
0,43.0,male,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
1,48.0,female,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
2,58.0,female,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
3,46.0,male,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,1,0,0,0
4,70.0,male,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
637,70.0,male,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
638,47.0,female,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
639,65.0,male,1,0,0,0,0,0,0,0,...,0,0,0,0,1,1,0,0,0,1
640,57.0,male,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,1


In [3]:
#DATA PREPROCESSING

In [4]:
#CHANGE VALUES IN THE gender COLUMN TO BINARY SO IT CAN BE USED IN LINEAR REG TO PREDICT MISSING AGE VALUES
#maleFemale = {'male': 1, 'female': 0}
#symptom_cd_dummies.gender = [maleFemale[item] for item in symptom_cd_dummies.gender]

In [5]:
x = symptom_cd_dummies.iloc[:, :-1].values
y = symptom_cd_dummies.iloc[:, -1].values

#LABEL ENCODE THE GENDER COLUMN
le = LabelEncoder()
x[:, 1] = le.fit_transform(x[:, 1])
#x[:,0]

In [6]:
#IMPUTE MISSING VALUES IN THE AGE COLUMN
imputer = KNNImputer(n_neighbors=2)

x = imputer.fit_transform(x)

x

array([[43.,  1.,  0., ...,  1.,  0.,  0.],
       [48.,  0.,  0., ...,  1.,  0.,  0.],
       [58.,  0.,  0., ...,  1.,  0.,  0.],
       ...,
       [65.,  1.,  1., ...,  0.,  0.,  0.],
       [57.,  1.,  0., ...,  0.,  0.,  0.],
       [77.,  0.,  0., ...,  1.,  0.,  0.]])

In [7]:
#SPLIT TRAIN AND TEST SET

In [8]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 0)

In [9]:
#FEATURE SCALING

In [10]:
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)

In [11]:
#LOAD SAVED MODEL

In [12]:
#LOAD JSON AND CREATE MODEL
json_file = open('model.json', 'r')
loaded_model_from_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_from_json)

#LOAD WEIGHTS INTO NEW MODEL
loaded_model.load_weights("model.h5")

#EVALUATE LOADED MODEL ON TEST DATA
loaded_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

y_pred = loaded_model.predict(x_test)
y_pred = (y_pred > 0.5)
compare = pd.DataFrame(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))
compare.head()

,0,1
0,0,0
1,0,0
2,0,0
3,0,0
4,0,0


In [13]:
#CONFUSION MATRIX

In [14]:
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[79  7]
 [14 29]]


0.8372093023255814

In [15]:
#PREDICTING THE OUTCOME OF A 65y/o MALE PATIENT WITH COPD
print(loaded_model.predict(sc.transform([[65,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]])) > 0.5)
print(loaded_model.predict(sc.transform([[65,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]])))

[[False]]
[[0.12076379]]


## Making a program that allows users to make predictions on a per patient basis based on input

In [16]:
from IPython.display import display, Markdown

#INITIALIZE LIST TO HOLD MATRIX OF FEATURES
mylist = list(range(34))

#INPUT PATIENT AGE AND GENDER
display(Markdown("### Demographics:"))
age = int(input("PROMPT 1: What is the patient's age? \n-\n"))
gender = input("\nPROMPT 2: What is the patient's sex? (write 'male' or 'female')\n-\n")
genderNum = 1 if gender=='male' else 0

#PROMPT USER INPUT OF SYMPTOMS
display(Markdown("### Health Profile:"))
display(Markdown("#### Symptoms:"))
print("2. acute respiratory distress   3. asymptomatic   4. cold chills   5. cough\n6. diarrhea   7. fatigue   8. fever.  9.headache.  10. malaise\n11. muscle soreness   12. pneumonia   13. runny nose   14. septic shock   15. shortness of breath\n16. sore throat")

#PROMPT USER INPUT OF PRE-EXISTING CONDITIONS
display(Markdown("#### Pre-existing Conditions:"))
print("17. tuberculosis   18.COPD   19.asthma   20. atherosclerosis   21. Cancer   22. cerebrovascular disease\n23. chronic bronchitis  24. diabetes   25. dyslipidemia   26. enlarged prostate   27. heart disease\n28. Hepatitis B   29. hypertension   30. kidney disease.  31. none   32. parkinson's disease.  33. thyroid disease\n")

symptoms = [int(x) for x in input("\nPROMPT 3: Enter the number next to the patient's corresponding symptoms, and/or pre-existing conditions as listed, \nseparated by a space:\n\n").split()]
mylist2 = [1 if int(el)==int(el) in symptoms else 0 for el in mylist]
  

mylist2[0] = age
mylist2[1] = genderNum

#RETURN THE PROBABILITY PREDICTION
probability = loaded_model.predict(sc.transform([mylist2]))
probabilityPercent = probability.round(2)

#RETURN THE PREDICTION IN FORM OF TRUE/FALSE
hardPrediction = probability>0.5

#RETURN THE 'TRUE/FALSE' VALUE OF 'hardPredction' WITHOUT THE BRACKETS
hardPrediction = ''.join(map(str, hardPrediction[0]))

class color:
   BOLD = '\033[1m'
   RED = '\033[91m'
   GREEN = '\033[92m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'

def final_prediction(finalResult):
    if (probability > 0.5):
        print(color.BOLD + hardPrediction + color.END)
        print("Our model predicts that a fatal infection is likely for this patient")
        print(color.RED + "HIGH RISK" + color.END)
    else:
        print(color.BOLD + hardPrediction + color.END)
        print("Our model predicts that a fatal infection is not likely for this patient")
        print(color.GREEN + "LOW RISK" + color.END)     

print(color.UNDERLINE + "\nPredicted probability of mortality:" + color.END)
print(*probability[0])
print(color.BOLD + "{0} %\n".format(*probabilityPercent[0]*100) + color.END)
print(color.UNDERLINE + "Fatal infection prediction:" + color.END)
final_prediction(hardPrediction)


### Demographics:

PROMPT 1: What is the patient's age? 
-
3

PROMPT 2: What is the patient's sex? (write 'male' or 'female')
-
male


### Health Profile:

#### Symptoms:

2. acute respiratory distress   3. asymptomatic   4. cold chills   5. cough
6. diarrhea   7. fatigue   8. fever.  9.headache.  10. malaise
11. muscle soreness   12. pneumonia   13. runny nose   14. septic shock   15. shortness of breath
16. sore throat


#### Pre-existing Conditions:

17. tuberculosis   18.COPD   19.asthma   20. atherosclerosis   21. Cancer   22. cerebrovascular disease
23. chronic bronchitis  24. diabetes   25. dyslipidemia   26. enlarged prostate   27. heart disease
28. Hepatitis B   29. hypertension   30. kidney disease.  31. none   32. parkinson's disease.  33. thyroid disease


PROMPT 3: Enter the number next to the patient's corresponding symptoms, and/or pre-existing conditions as listed, 
separated by a space:

3

Predicted probability of mortality:
0.005161185
1.0 %

Fatal infection prediction:
False
Our model predicts that a fatal infection is not likely for this patient
LOW RISK


In [22]:
#INITIALIZE LIST TO HOLD MATRIX OF FEATURES
mylist = list(range(34))



class color:
   BOLD = '\033[1m'
   RED = '\033[91m'
   GREEN = '\033[92m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'


while True:
    
    try:
        #INPUT PATIENT AGE AND GENDER
        display(Markdown("### Demographics:"))
        age = int(input("PROMPT 1: What is the patient's age? \n-\n"))
    except ValueError:
        print('invalid input, please enter a number.')
        continue
    else:
        
        while True:
            gender = input("\nPROMPT 2: What is the patient's sex? (write 'male' or 'female')\n-\n")
            genderNum = 1 if gender=='male' else 0

            if gender not in ('male', 'female'):
                print('invalid input, please write "male" or "female".')
                continue
            else:
                
                while True:
                    #PROMPT USER INPUT OF SYMPTOMS
                    display(Markdown("### Health Profile:"))
                    display(Markdown("#### Symptoms:"))
                    print("2. acute respiratory distress   3. asymptomatic   4. cold chills   5. cough\n6. diarrhea   7. fatigue   8. fever.  9.headache.  10. malaise\n11. muscle soreness   12. pneumonia   13. runny nose   14. septic shock   15. shortness of breath\n16. sore throat")

                    #PROMPT USER INPUT OF PRE-EXISTING CONDITIONS
                    display(Markdown("#### Pre-existing Conditions:"))
                    print("17. tuberculosis   18.COPD   19.asthma   20. atherosclerosis   21. Cancer   22. cerebrovascular disease\n23. chronic bronchitis  24. diabetes   25. dyslipidemia   26. enlarged prostate   27. heart disease\n28. Hepatitis B   29. hypertension   30. kidney disease.  31. none   32. parkinson's disease.  33. thyroid disease\n")
                    
                    try:
                        symptoms = list(map(int,input("\nPROMPT 3: Enter the number next to the patient's corresponding symptoms, and/or pre-existing conditions as listed, \nseparated by a space:\n\n").split()))
                        mylist2 = [1 if int(el)==int(el) in symptoms else 0 for el in mylist]
                    except ValueError:
                        print('invalid, please enter a number')
                        continue
                    else:
                        
                        mylist2[0] = age
                        mylist2[1] = genderNum

                        #RETURN THE PROBABILITY PREDICTION
                        probability = loaded_model.predict(sc.transform([mylist2]))
                        probabilityPercent = probability.round(2)

                        #RETURN THE PREDICTION IN FORM OF TRUE/FALSE
                        hardPrediction = probability>0.5

                        #RETURN THE 'TRUE/FALSE' VALUE OF 'hardPredction' WITHOUT THE BRACKETS
                        hardPrediction = ''.join(map(str, hardPrediction[0]))

                        print(color.UNDERLINE + "\nPredicted probability of mortality:" + color.END)
                        print(*probability[0])
                        print(color.BOLD + "{0} %\n".format(*probabilityPercent[0]*100) + color.END)
                        print(color.UNDERLINE + "Fatal infection prediction:" + color.END)

                        if (probability > 0.5):
                            print(color.BOLD + hardPrediction + color.END)
                            print("Our model predicts that a fatal infection is likely for this patient")
                            print(color.RED + "HIGH RISK" + color.END)
                        else:
                            print(color.BOLD + hardPrediction + color.END)
                            print("Our model predicts that a fatal infection is not likely for this patient")
                            print(color.GREEN + "LOW RISK" + color.END)
                        break
                break
                    
    again = input('\nEnter new patient info? (y/n): ')
    if again not in ('y', 'n'):
        print('Invalid input, please type "y" or "n".')
        continue
    elif again == 'y':
        continue
    else:
        print('Thank you.')
        break
    break

### Demographics:

PROMPT 1: What is the patient's age? 
-
24

PROMPT 2: What is the patient's sex? (write 'male' or 'female')
-
male


### Health Profile:

#### Symptoms:

2. acute respiratory distress   3. asymptomatic   4. cold chills   5. cough
6. diarrhea   7. fatigue   8. fever.  9.headache.  10. malaise
11. muscle soreness   12. pneumonia   13. runny nose   14. septic shock   15. shortness of breath
16. sore throat


#### Pre-existing Conditions:

17. tuberculosis   18.COPD   19.asthma   20. atherosclerosis   21. Cancer   22. cerebrovascular disease
23. chronic bronchitis  24. diabetes   25. dyslipidemia   26. enlarged prostate   27. heart disease
28. Hepatitis B   29. hypertension   30. kidney disease.  31. none   32. parkinson's disease.  33. thyroid disease


PROMPT 3: Enter the number next to the patient's corresponding symptoms, and/or pre-existing conditions as listed, 
separated by a space:

2 24

Predicted probability of mortality:
0.7956117
80.0 %

Fatal infection prediction:
True
Our model predicts that a fatal infection is likely for this patient
HIGH RISK

Enter new patient info? (y/n): n
Thank you.


In [24]:
#mylist2